In [1]:
import json
import pandas as pd

# Part 1: Reading and investigating data

Open up the `player_data.json` and investigate the data structure.

In [2]:
import sys
#your code here
data = json.loads(open('player_data.json').read())
data[0].keys()

Pandas Challenge NBA.ipynb player_data.json


# Part 2:  Creating the Dataframe

Create a data frame where each row corresponds to a game for each player. You must have atleast the following columns in your final dataframe:

'DISPLAY_FIRST_LAST', 'PERSON_ID', 'TEAM_ID', 'AST', 'BLK', 'DREB','Game_ID',  'MIN', 'PTS', 'REB', 'TEAM_ABBREVIATION'

*You can include additional columns in your dataframe, so you don't have to do additional work to remove other columns.*  

In [3]:
import sys
#columns = ['DISPLAY_FIRST_LAST', 'PERSON_ID', 'TEAM_ID', 'AST', 'STL', 'BLK', 'DREB','Game_ID', 'MIN', 'PTS', 'REB', 'TEAM_ABBREVIATION']
df = pd.DataFrame()

for person in data:

    if len(person['GAMELOG']) == 0:
        continue
    df1 = pd.DataFrame(person, columns=['DISPLAY_FIRST_LAST', 'PERSON_ID', 'TEAM_ID', 'CAREER_STATS'], index=[0])
    df2 = pd.DataFrame(person['GAMELOG'])
    merged = pd.merge(df1, df2, left_on='PERSON_ID', right_on='Player_ID', how='outer')
    df = pd.concat([df, merged])
    
df.reset_index(drop=True, inplace=True)

# Part 3: Pandas Questions

Answer the following questions

## 1 
- Find the player who scored the most points in an individual game last season.

-  Now do the same for rebounds, blocks, and assists.

In [4]:
# Your code here
max_points_game_df = df.loc[df['PTS'] == df['PTS'].max()]
df['REB'] = df['OREB'] + df['DREB']
max_rebs_game_df = df[df['REB'] == df['REB'].max()]
max_blks_game_df = df[df['BLK'] == df['BLK'].max()]
max_asts_game_df = df[df['AST'] == df['AST'].max()]
pd.concat([max_asts_game_df, max_blks_game_df, max_rebs_game_df,max_points_game_df])['DISPLAY_FIRST_LAST'].unique()


array(['Russell Westbrook', 'Mitchell Robinson', 'Hassan Whiteside',
       'Karl-Anthony Towns', 'James Harden'], dtype=object)

## 2: 

- Find the player who scored the most points for the entire season.

- Now do the same for rebounds, blocks, and assists.

In [178]:
import numpy as np
#your code here
df_to_group = df[['PERSON_ID', 'AST','BLK', 'PTS', 'REB', 'STL']]
gb_player = df_to_group.groupby('PERSON_ID').sum()
df[df['PERSON_ID'] == np.argmax(gb_player['PTS'])]['DISPLAY_FIRST_LAST'].iloc[0]

'James Harden'

## 3:
How many players played in at least 82 games last season?

In [181]:
#your code here
len(df.groupby('PERSON_ID').filter(lambda x: x['Game_ID'].count() >= 82).groupby('DISPLAY_FIRST_LAST').first().index.tolist())

21

## 4: 
- Find which team scored the most points for the entire season.

- Now do the same for rebounds, blocks, and assists.

In [185]:
# your code here
team_data = df[['TEAM_ABBREVIATION', 'AST','BLK', 'PTS', 'REB', 'STL']]
team_groups = team_data.groupby('TEAM_ABBREVIATION').sum()
max_vals = team_data.groupby('TEAM_ABBREVIATION').agg(lambda x: sum(x)).agg(lambda x: max(x)).reset_index()
#pd.merge(team_groups, max_vals, left_on)
unstacked_teams = team_groups.unstack().reset_index()
pd.merge(unstacked_teams, max_vals)


,level_0,TEAM_ABBREVIATION,0,index
0,AST,GSW,2413,AST
1,BLK,GSW,525,BLK
2,PTS,MIL,9686,PTS
3,REB,MIL,4078,REB
4,STL,OKC,766,STL


## 5:
Which team had the most players play for them last season?

In [218]:
#your code here
n_players = df.groupby('TEAM_ABBREVIATION')['PERSON_ID'].nunique()
n_players.loc[n_players.apply(lambda x: x >= n_players.max())]

TEAM_ABBREVIATION
MEM    28
Name: PERSON_ID, dtype: int64

## 6:
- Did any players play for more than one team last year?

- What is the most number of teams a player played for last season?

In [153]:
n_teams_by_player = df.groupby('PERSON_ID')['TEAM_ABBREVIATION'].nunique()
print(n_teams_by_player.apply(lambda x: x > 1).sum())
n_teams_by_player.max()

86


3

## 7:

Find the player who scored the least amount of points while playing in at least 40 games last season.

In [195]:
#your code here
player_totals_df = df.groupby('DISPLAY_FIRST_LAST').sum()
player_totals_df.loc[player_totals_df['PTS'] == df.groupby('PERSON_ID').sum().loc[df.groupby('PERSON_ID')['Game_ID'].count().apply(lambda x: x >= 40)]['PTS'].min()]#.reset_index()['DISPLAY_FIRST_LAST']


,PERSON_ID,TEAM_ID_x,AST,BLK,DREB,FG3A,FG3M,FG3_PCT,FGA,FGM,...,OREB,PF,PLUS_MINUS,PTS,Player_ID,REB,STL,TEAM_ID_y,TOV,VIDEO_AVAILABLE
DISPLAY_FIRST_LAST,,,,,,,,,,,,,,,,,,,,,
Sindarius Thornwell,104218496,103079215744,18,7,39,15,3,2.333,49,17,...,5,41,-38,62,104218496,44,14,103079215744,20,65


# Part 4: Pandas and Probability

## 8: 

What is the probability that a random player had a game where they scored more than 40 points?

In [215]:
#your code here
df.loc[df['PTS'].apply(lambda x: x > 40)].count()['Game_ID']/df.count()['GAME_DATE']#.count()/df.count()


0.004214397915788667

## 9:

What is the probability that a randomly selected player from last season would average more than 20 points per game?

In [221]:
# your code here
len(df.groupby('DISPLAY_FIRST_LAST').mean().loc[df.groupby('DISPLAY_FIRST_LAST').mean()['PTS'].apply(lambda x: x>20)])/len(df.groupby('DISPLAY_FIRST_LAST').mean())

0.062264150943396226

# Advanced Questions

## 10: 
Find the player who scored the least amount of points while playing in at least 40 games and averageing at least 15 minutes per game last season.

In [15]:
#your code here
#your code here
player_totals_df = df.groupby('DISPLAY_FIRST_LAST').sum()
played_40 = df.groupby('DISPLAY_FIRST_LAST').count()['Game_ID'].apply(lambda x: x>=40)
played_15 = df.groupby('DISPLAY_FIRST_LAST').mean()['MIN'].apply(lambda x: x>= 15)
games_minutes_df = pd.merge(played_40, played_15, left_index=True, right_index=True).reset_index()
games_minutes_df = games_minutes_df.loc[games_minutes_df['Game_ID'] & games_minutes_df['MIN']]
totals_with_40_15 = pd.merge(games_minutes_df, player_totals_df, on='DISPLAY_FIRST_LAST', how='left')
totals_with_40_15.iloc[totals_with_40_15['PTS'].idxmin(axis=1)]
#player_totals_df.loc[player_totals_df['PTS'] == df.groupby('PERSON_ID').sum().loc[df.groupby('PERSON_ID')['Game_ID'].count().apply(lambda x: x >= 40)]['PTS'].min()]#.reset_index()['DISPLAY_FIRST_LAST']


DISPLAY_FIRST_LAST    Tyson Chandler
Game_ID                         True
MIN_x                           True
PERSON_ID                     120945
TEAM_ID_x                88583700975
CAREER_STATS                       0
AST                               37
BLK                               23
DREB                             215
FG3A                               1
FG3M                               0
FG3_PCT                            0
FGA                               99
FGM                               61
FG_PCT                        27.563
FTA                               87
FTM                               51
FT_PCT                        18.966
MIN_y                            876
OREB                              92
PF                               110
PLUS_MINUS                       -36
PTS                              173
Player_ID                     120945
REB                              307
STL                               21
TEAM_ID_y                88583701148
T

## 11: 
Which player scored the largest share of points for their team throughout the season.

In [ ]:
#your code here

## 12:

What is the probability that a randomly selected player had at least one game in which they scored 40 points last season. 

In [ ]:
# your code here

## 13: 

A double-double is when a player records for any of the two follow categories: points, rebounds, assists, steals, or blocks. 
What is the probability that a randomly selected player had a double-double in a game last season?


In [272]:
#your code here
# Use the height and width to calculate the area
def is_double_double(row):
    return sum([value > 9 for key, value in row.items() if key in ('REB', 'AST', 'PTS','STl','BLK')]) > 1

df['is_double_double'] = df.apply(is_double_double, axis=1)
df.groupby('DISPLAY_FIRST_LAST').agg({'is_double_double':'any'})['is_double_double'].sum()/df['DISPLAY_FIRST_LAST'].nunique()


0.49056603773584906

## 14:

What is the probability that a randomnly selected game had a player record a triple-double in that game?

In [273]:
#your code here
# Use the height and width to calculate the area
def is_triple_double(row):
    return sum([value > 9 for key, value in row.items() if key in ('REB', 'AST', 'PTS','STl','BLK')]) > 2

df['is_triple_double'] = df.apply(is_triple_double, axis=1)
df.groupby('DISPLAY_FIRST_LAST').agg({'is_triple_double':'any'})['is_triple_double'].sum()/df['DISPLAY_FIRST_LAST'].nunique()


0.06981132075471698

# Part 5: Super Duper Challenge



## 15:

How many players last season averaged more points per game than their career average?

In [239]:
df.groupby('DISPLAY_FIRST_LAST').agg({'PTS':sum,'CAREER_STATS':'first'})

,PTS,CAREER_STATS
DISPLAY_FIRST_LAST,,
Aaron Gordon,1246,NaN
Aaron Holiday,294,NaN
Abdel Nader,241,NaN
Al Horford,925,NaN
Al-Farouq Aminu,760,NaN
Alan Williams,18,NaN
Alec Burks,561,NaN
Alex Abrines,165,NaN
Alex Caruso,229,NaN


In [ ]:
df.